# Predict the Phasemaks

In [ ]:
import os
import shutil
import subprocess
import re

def clear_directory(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory)

def extract_index(filename):
    match = re.search(r'(\d+)', filename)
    if match:
        return int(match.group(1))
    return float('inf')  # Use infinity to push non-matching filenames to the end


def move_and_predict(camera1_dir, camera2_dir, phasemask_dir, prediction_dir, prediction_script, output_dir_template):
    beam_ff_dir = os.path.join(prediction_dir, 'beam_ff')
    beam_nf_dir = os.path.join(prediction_dir, 'beam_nf')
    phasemask_predict_dir = os.path.join(prediction_dir, 'phasemask')

    os.makedirs(prediction_dir, exist_ok=True)
    os.makedirs(beam_ff_dir, exist_ok=True)
    os.makedirs(beam_nf_dir, exist_ok=True)
    os.makedirs(phasemask_predict_dir, exist_ok=True)

    camera1_files = sorted(os.listdir(camera1_dir), key=extract_index)
    camera2_files = sorted(os.listdir(camera2_dir), key=extract_index)
    phasemask_files = sorted(os.listdir(phasemask_dir), key=extract_index)

    index = 1

    for camera1_file, camera2_file, phasemask_file in zip(camera1_files, camera2_files, phasemask_files):
        clear_directory(beam_ff_dir)
        clear_directory(beam_nf_dir)
        clear_directory(phasemask_predict_dir)

        shutil.copy(os.path.join(camera2_dir, camera2_file), os.path.join(beam_ff_dir, camera2_file))
        shutil.copy(os.path.join(camera1_dir, camera1_file), os.path.join(beam_nf_dir, camera1_file))
        shutil.copy(os.path.join(phasemask_dir, phasemask_file), os.path.join(phasemask_predict_dir, phasemask_file))

        try:
            result = subprocess.run(['python', prediction_script], check=True, capture_output=True, text=True)
            print(result.stdout)
        except subprocess.CalledProcessError as e:
            print(f"Error occurred while running the prediction script: {e.stderr}")
            break
        
        predicted_image_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\phasemask_predicted.png'
        if os.path.exists(predicted_image_path):
            new_predicted_image_path = output_dir_template.format(index)
            shutil.move(predicted_image_path, new_predicted_image_path)
        
        index += 1
        print(f"Processed {index} predictions")

# Vortex unwrapped
# camera1_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datasets\Dataset_Vortex_20000imgs_shuffle_cleaned_v2_unwrapped_final\test\beam_nf'
# camera2_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datasets\Dataset_Vortex_20000imgs_shuffle_cleaned_v2_unwrapped_final\test\beam_ff'
# phasemask_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datasets\Dataset_Vortex_20000imgs_shuffle_cleaned_v2_unwrapped_final\test\phasemask'
# output_dir_template = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\Test_Random_200_phasemasks\MSE_all_test_images\phasemask_predicted_unwrapped\phasemask_predicted_{}.png'

# Vortex Test
# camera1_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\Test_new_collected_200Testimages\test\camera_1_crop_test'
# camera2_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\Test_new_collected_200Testimages\test\camera_2_crop_test'
# phasemask_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\Test_new_collected_200Testimages\test\phasemask_crop_test'
# output_dir_template = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\Test_new_collected_200Testimages\test\phasemask_predicted_unwrapped_test\phasemask_predicted_{}.png'

# Gaussian Evaluation 1
# camera1_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Gaussian\camera_1_abb_noise_crop'
# camera2_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Gaussian\camera_2_abb_noise_crop'
# phasemask_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Gaussian\phasemask_unwrapped_crop_index'
# output_dir_template = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Gaussian\phasemask_unwrapped_prediction\phasemask_predicted_{}.png'

# # Gaussian Evaluation 2
# camera1_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Gaussian_2\camera_1_crop'
# camera2_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Gaussian_2\camera_2_crop'
# phasemask_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Gaussian_2\phasemask_unwrapped_crop_index'
# output_dir_template = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Gaussian_2\phasemask_unwrapped_predicted\phasemask_predicted_{}.png'


# # Gaussian Test
# camera1_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datasets\Dataset_Gaussian_real_ditzingen_Abb08_shuffle_cleaned_unwrapped\test\beam_nf'
# camera2_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datasets\Dataset_Gaussian_real_ditzingen_Abb08_shuffle_cleaned_unwrapped\test\beam_ff'
# phasemask_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Datasets\Dataset_Gaussian_real_ditzingen_Abb08_shuffle_cleaned_unwrapped\test\phasemask'
# output_dir_template = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Gaussian\Test\test_prediction\phasemask_predicted_{}.png'

# Schramberg Test
# camera1_dir = r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Evaluation\camera_1'
# camera2_dir = r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Evaluation\camera_2'
# phasemask_dir = r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Evaluation\phasemask'
# output_dir_template = r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Evaluation\phasemask_prediction\phasemask_predicted_{}.png'

# Vortex Test
# camera1_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\Test_Random_200_phasemasks\camera_1_crop'
# camera2_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\Test_Random_200_phasemasks\camera_2_crop'
# phasemask_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\Test_Random_200_phasemasks\phasemask_unwrapped_crop_index'
# output_dir_template = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\Test_Random_200_phasemasks\Pretiction_Test_schramberg\phasemask_predicted_{}.png'

# Schramberg
# camera1_dir = r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Evaluation\camera_1'
# camera2_dir = r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Evaluation\camera_2'
# phasemask_dir = r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Evaluation\phasemask'
# output_dir_template = r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Evaluation_2\phasemask_prediction\phasemask_predicted_{}.png'

# Referenz image
# camera1_dir = r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Test\camera_1_crop'
# camera2_dir = r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Test\camera_2_Crop'
# phasemask_dir = r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Test\phasemask'
# output_dir_template = r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Test\predicted_phasemask\phasemask_predicted_{}.png'

# Test
# camera1_dir = r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Evaluation_Kopie\camera_1'
# camera2_dir = r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Evaluation_Kopie\camera_2'
# phasemask_dir = r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Evaluation_Kopie\phasemask'
# output_dir_template = r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Evaluation_Kopie\phasemask_prediction_{}.png'

# Predict kapitel
#gaussian unwrapped image 1
# camera1_dir = r'D:\Masterarbeit\Evaluation\Prediction\experimental_unwrapped\gaussian_img1\camera_1'
# camera2_dir = r'D:\Masterarbeit\Evaluation\Prediction\experimental_unwrapped\gaussian_img1\camera_2'
# phasemask_dir = r'D:\Masterarbeit\Evaluation\Prediction\experimental_unwrapped\gaussian_img1\phasemask'
# output_dir_template = r'D:\Masterarbeit\Evaluation\Prediction\experimental_unwrapped\gaussian_img1\predicted_phasemask\phasemask_prediction_{}.png'

# camera1_dir = r'\\srvditz1\home$\burckhardsv\Desktop\MSE calculation\gaussian_img1\camera_1'
# camera2_dir = r'\\srvditz1\home$\burckhardsv\Desktop\MSE calculation\gaussian_img1\camera_2'
# phasemask_dir = r'\\srvditz1\home$\burckhardsv\Desktop\MSE calculation\gaussian_img1\phasemask'
# output_dir_template = r'\\srvditz1\home$\burckhardsv\Desktop\MSE calculation\gaussian_img1\predicted_phasemask\phasemask_prediction_{}.png'


# gaussian unwrapped image 2
# camera1_dir = r'D:\Masterarbeit\Evaluation\Prediction\experimental_unwrapped\gaussian_img2\camera_1'
# camera2_dir = r'D:\Masterarbeit\Evaluation\Prediction\experimental_unwrapped\gaussian_img2\camera_2'
# phasemask_dir = r'D:\Masterarbeit\Evaluation\Prediction\experimental_unwrapped\gaussian_img2\phasemask'
# output_dir_template = r'D:\Masterarbeit\Evaluation\Prediction\experimental_unwrapped\gaussian_img2\predicted_phasemask\phasemask_prediction_{}.png'

camera1_dir = r'\\srvditz1\home$\burckhardsv\Desktop\MSE calculation\gaussian_img2\camera_1'
camera2_dir = r'\\srvditz1\home$\burckhardsv\Desktop\MSE calculation\gaussian_img2\camera_2'
phasemask_dir = r'\\srvditz1\home$\burckhardsv\Desktop\MSE calculation\gaussian_img2\phasemask'
output_dir_template = r'\\srvditz1\home$\burckhardsv\Desktop\MSE calculation\gaussian_img1\predicted_phasemask\phasemask_prediction_{}.png'

# Vortex unwrapped image 1
# camera1_dir = r'D:\Masterarbeit\Evaluation\Prediction\experimental_unwrapped\vortex_img1\camera_1'
# camera2_dir = r'D:\Masterarbeit\Evaluation\Prediction\experimental_unwrapped\vortex_img1\camera_2'
# phasemask_dir = r'D:\Masterarbeit\Evaluation\Prediction\experimental_unwrapped\vortex_img1\phasemask'
# output_dir_template = r'D:\Masterarbeit\Evaluation\Prediction\experimental_unwrapped\vortex_img1\predicted_phasemask\phasemask_prediction_{}.png'

# camera1_dir = r'\\srvditz1\home$\burckhardsv\Desktop\MSE calculation\vortex_img1\camera_1'
# camera2_dir = r'\\srvditz1\home$\burckhardsv\Desktop\MSE calculation\vortex_img1\camera_2'
# phasemask_dir = r'\\srvditz1\home$\burckhardsv\Desktop\MSE calculation\vortex_img1\phasemask'
# output_dir_template = r'\\srvditz1\home$\burckhardsv\Desktop\MSE calculation\vortex_img1\predicted_phasemask\phasemask_prediction_{}.png'

# Vortesx unwrapped image 2
# camera1_dir = r'D:\Masterarbeit\Evaluation\Prediction\experimental_unwrapped\vortex_img2\camera_1'
# camera2_dir = r'D:\Masterarbeit\Evaluation\Prediction\experimental_unwrapped\vortex_img2\camera_2'
# phasemask_dir = r'D:\Masterarbeit\Evaluation\Prediction\experimental_unwrapped\vortex_img2\phasemask'
# output_dir_template = r'D:\Masterarbeit\Evaluation\Prediction\experimental_unwrapped\vortex_img2\predicted_phasemask\phasemask_prediction_{}.png'

# camera1_dir = r'\\srvditz1\home$\burckhardsv\Desktop\MSE calculation\vortex_img2\camera_1'
# camera2_dir = r'\\srvditz1\home$\burckhardsv\Desktop\MSE calculation\vortex_img2\camera_2'
# phasemask_dir = r'\\srvditz1\home$\burckhardsv\Desktop\MSE calculation\vortex_img2\phasemask'
# output_dir_template = r'\\srvditz1\home$\burckhardsv\Desktop\MSE calculation\vortex_img2\predicted_phasemask\phasemask_prediction_{}.png'


# always the same
prediction_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\predict_512'
prediction_script = r'C:\Users\burckhardsv\Lokale_Dateien\Playground.schwartzma\Beam_Shaping_using_ML\experimental_sven\Autoencoder\prediction.py'

move_and_predict(camera1_dir, camera2_dir, phasemask_dir, prediction_dir, prediction_script, output_dir_template)

### Code with debug prints

In [ ]:
# import os
# import shutil
# import subprocess
# import re

# def clear_directory(directory):
#     if os.path.exists(directory):
#         shutil.rmtree(directory)
#     os.makedirs(directory)

# def extract_index(filename):
#     match = re.search(r'(\d+)', filename)
#     if match:
#         return int(match.group(1))
#     return float('inf')  # Use infinity to push non-matching filenames to the end

# def move_and_predict(camera1_dir, camera2_dir, phasemask_dir, prediction_dir, prediction_script, output_dir_template):
#     beam_ff_dir = os.path.join(prediction_dir, 'beam_ff')
#     beam_nf_dir = os.path.join(prediction_dir, 'beam_nf')
#     phasemask_predict_dir = os.path.join(prediction_dir, 'phasemask')

#     os.makedirs(prediction_dir, exist_ok=True)
#     os.makedirs(beam_ff_dir, exist_ok=True)
#     os.makedirs(beam_nf_dir, exist_ok=True)
#     os.makedirs(phasemask_predict_dir, exist_ok=True)

#     camera1_files = sorted(os.listdir(camera1_dir), key=extract_index)
#     camera2_files = sorted(os.listdir(camera2_dir), key=extract_index)
#     phasemask_files = sorted(os.listdir(phasemask_dir), key=extract_index)

#     # Debug output for sorted files
#     print("Sorted camera1 files:", camera1_files)
#     print("Sorted camera2 files:", camera2_files)
#     print("Sorted phasemask files:", phasemask_files)

#     index = 1

#     for camera1_file, camera2_file, phasemask_file in zip(camera1_files, camera2_files, phasemask_files):
#         # Debug output for current files
#         print(f"Processing: Camera1 File: {camera1_file}, Camera2 File: {camera2_file}, Phasemask File: {phasemask_file}")
#         print(f"Indices: Camera1: {extract_index(camera1_file)}, Camera2: {extract_index(camera2_file)}, Phasemask: {extract_index(phasemask_file)}")

#         clear_directory(beam_ff_dir)
#         clear_directory(beam_nf_dir)
#         clear_directory(phasemask_predict_dir)

#         shutil.copy(os.path.join(camera2_dir, camera2_file), os.path.join(beam_ff_dir, camera2_file))
#         shutil.copy(os.path.join(camera1_dir, camera1_file), os.path.join(beam_nf_dir, camera1_file))
#         shutil.copy(os.path.join(phasemask_dir, phasemask_file), os.path.join(phasemask_predict_dir, phasemask_file))

#         try:
#             result = subprocess.run(['python', prediction_script], check=True, capture_output=True, text=True)
#             print(result.stdout)
#         except subprocess.CalledProcessError as e:
#             print(f"Error occurred while running the prediction script: {e.stderr}")
#             break
        
#         predicted_image_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\phasemask_predicted.png'
#         if os.path.exists(predicted_image_path):
#             new_predicted_image_path = output_dir_template.format(index)
#             shutil.move(predicted_image_path, new_predicted_image_path)
        
#         index += 1
#         print(f"Processed {index} predictions")

# # Define folders
# camera1_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\Test_Random_200_phasemasks\camera_1_crop'
# camera2_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\Test_Random_200_phasemasks\camera_2_crop'
# phasemask_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\Test_Random_200_phasemasks\phasemask_unwrapped_crop_index'
# output_dir_template = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\Test_Random_200_phasemasks\phasemask_unwrapped_predict\phasemask_predicted_{}.png'

# # camera1_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\Test_new_collected_200Testimages\test\camera_1_crop_test'
# # camera2_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\Test_new_collected_200Testimages\test\camera_2_crop_test'
# # phasemask_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\Test_new_collected_200Testimages\test\phasemask_crop_test'
# # output_dir_template = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\Test_new_collected_200Testimages\test\phasemask_predicted_unwrapped_test\phasemask_predicted_{}.png'

# # camera1_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\Test_new_collected_200Testimages\test\camera_1_crop'
# # camera2_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\Test_new_collected_200Testimages\test\camera_2_crop'
# # phasemask_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\Test_new_collected_200Testimages\test\phasemask_crop'
# # output_dir_template = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex\Test_new_collected_200Testimages\test\phasemask_predicted_unwrapped\phasemask_predicted_{}.png'

# # Always the same
# prediction_dir = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\predict_512'
# prediction_script = r'C:\Users\burckhardsv\Lokale_Dateien\Playground.schwartzma\Beam_Shaping_using_ML\experimental_sven\Autoencoder\prediction.py'

# move_and_predict(camera1_dir, camera2_dir, phasemask_dir, prediction_dir, prediction_script, output_dir_template)


# Create original Phasemak Size

In [2]:
from PIL import Image
import numpy as np
import numpy as np
import cv2
from matplotlib import pyplot as plt
from skimage.color import rgb2gray

In [3]:
def uncrop_to_original(original_phasemask_path, cropped_image_path, save_path, crop_width=512, crop_height=512, rgb_value=0): #rgb value before 128
    original_image = Image.open(original_phasemask_path)
    cropped_image = Image.open(cropped_image_path)
    
    original_image_np = np.array(original_image)
    cropped_image_np = np.array(cropped_image)
    
    center_y, center_x = original_image_np.shape[0] // 2, original_image_np.shape[1] // 2

    start_x = max(center_x - crop_width // 2, 0)
    start_y = max(center_y - crop_height // 2, 0)
    
    end_x = min(start_x + crop_width, original_image_np.shape[1])
    end_y = min(start_y + crop_height, original_image_np.shape[0])
    
    new_image_np = np.full_like(original_image_np, rgb_value)
    
    new_image_np[start_y:end_y, start_x:end_x] = cropped_image_np
    
    new_image = Image.fromarray(new_image_np)
    
    new_image.save(save_path)
    
    return new_image


In [4]:
# checked
import os

# Define paths
original_phasemask_path = r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\phase_unwrapped_index\1_idx_1.png' # can be one phasemask doesn't matter but needs the original size
predicted_phasemask_folder = r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Test\predicted_phasemask'
output_folder = r'C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Test\predicted_phasemaskoriginal_size'

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Loop through all files in the predicted images folder
for file_name in os.listdir(predicted_phasemask_folder):
    predicted_phasemask_path = os.path.join(predicted_phasemask_folder, file_name)
    if os.path.isfile(predicted_phasemask_path):
        save_path = os.path.join(output_folder, file_name.replace('phasemask_predicted_', 'predicted_phasemask_original_size_'))
        uncrop_to_original(original_phasemask_path, predicted_phasemask_path, save_path, rgb_value=0)
        print(f"Processed {file_name} and saved as {save_path}")

print("All images have been processed and resized to original size.")

Processed phasemask_predicted_1.png and saved as C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Test\predicted_phasemaskoriginal_size\predicted_phasemask_original_size_1.png
All images have been processed and resized to original size.


# Create Compensated Phasemask

In [7]:
import cv2
import imageio.v2 as iio
import matplotlib.pyplot as plt
import numpy as np
from LightPipes import *
import math
import os
import glob
 
# Define base paths and file naming conventions
#base_path_original = r"\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Gaussian\phasemask_unwrapped_original_index" # 
# base_path_original = r"C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Evaluation\phasemask_unwrapped_original_index_and_size" # unwrapped_original with index original size
# base_path_predicted = r"C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Evaluation\phasemask_prediction_original_size"
# output_base_path = r"C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Evaluation\phasemask_compensated"

#Schramberg evaluation
# base_path_original = r"C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Evaluation_2\phasemask_unwrapped_original_index_and_size" # unwrapped_original with index original size
# base_path_predicted = r"C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Evaluation_2\phasemask_prediction_index_original_size"
# output_base_path = r"C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Evaluation_2\phasemask_compensated"

#Schramberg perfect Phasemask
# base_path_original = r"C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Evaluation_2\phasemask_unwrapped_original_index_and_size" # unwrapped_original with index original size
# base_path_predicted = r"C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Evaluation_2\phasemask_prediction_index_original_size"
# output_base_path = r"C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Schramberg_Datacollection\Schramberg_Datacollection\Evaluation_2\phasemask_compensated"


# Test the correction of the beam without slm
# base_path_original = r"C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Test\original_phasemask_with zeros" # unwrapped_original with index original size
# base_path_predicted = r"C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Test\predicted_phasemaskoriginal_size"
# output_base_path = r"C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Test\compansated_phasemask"
 
# List of indices to process#
#indices = [1, 2, 3, 4, 5, 6,7,8,9,10,11,12]  # Add more indices as needed
indices = list(range(1,2))
 
for idx in indices:
    # Search for files with the given index
    original_files = glob.glob(os.path.join(base_path_original, f"{idx}_*.png"))
    predicted_files = glob.glob(os.path.join(base_path_predicted, f"{idx}_*.png"))
 
    if original_files and predicted_files:
        original_file = original_files[0]
        predicted_file = predicted_files[0]
        output_file = os.path.join(output_base_path, f"{idx}_idx_compensated_phase.png")
 
        # read an image
        print(f'{original_file} vs {predicted_file}' )
        img_orginal = iio.imread(original_file)
        img_pre = iio.imread(predicted_file)
       
        # Change the phase profile from gray value to phase value with the original phase mask
        phase_orginal = 16 / 255 * (img_orginal)  # change the gray value to 16
        T = np.exp(1j * phase_orginal)
       
        # Change the phase profile from gray value to phase value with the predicted phase mask
        phase_pre = 16 / 255 * (img_pre)  # change the gray value to 16
        T = np.exp(-1j * phase_pre)
       
        # Generate the transmitted function and also the wrapped phase function to display on SLM
        T = np.exp(1j * phase_orginal) * np.exp(-1j * phase_pre)
        Phase_compensation = np.angle(T)
       
        # Save the compensated phase image
        plt.imsave(output_file, Phase_compensation, cmap='gray')
 
print("Processing complete.")

C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Test\original_phasemask_with zeros\1_idx_test.png vs C:\Users\burckhardsv\Lokale_Dateien\Schramberg\Test\predicted_phasemaskoriginal_size\1_idx_predicted_phasemask_original_size_1.png
Processing complete.


# ABLAGE

gaussian test

In [ ]:
predicted_phasemask_original_size_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Gaussian\Abb08\results_phasemask_2\predicted_phasemask_original_size.png'
original_phasemask_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Gaussian\Abb08\phasemask\phase_mask_2.png'

# predicted_phasemask_original_size_path= r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven%20Burckhard\Predict_Phasemask\Evaluation_Vortex_Wrong_Phasemasks\evaluation_phasemask_predicted_original_size\predicted_phasemask_original_size_2.png'
# original_phasemask_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex_Wrong_Phasemasks\phasemask\phase_mask_2.png'

predicted_phasemask_original_size = cv2.imread(predicted_phasemask_original_size_path)
original_phasemask = cv2.imread(original_phasemask_path)



predicted_phasemask_original_size = rgb2gray(predicted_phasemask_original_size)
original_phasemask = rgb2gray(original_phasemask)


plt.title("Beam_cropped")
im=plt.imshow(predicted_phasemask_original_size,cmap='gray')
plt.colorbar(im)
plt.show()

plt.title("Beam_cropped")
im=plt.imshow(original_phasemask,cmap='gray')
plt.colorbar(im)
plt.show()

print(predicted_phasemask_original_size.max())
print(original_phasemask.max())
print(predicted_phasemask_original_size.min())
print(original_phasemask.min())

compensate_phasemask = abs(original_phasemask - predicted_phasemask_original_size)
#compensate_phasemask = original_phasemask - predicted_phasemask_original_size

# print(compensate_phasemask.max())
# print(compensate_phasemask.min())
#print(compensate_phasemask)

print(compensate_phasemask)

plt.title("compensate_phasemask")
im=plt.imshow(compensate_phasemask,cmap='gray')
plt.colorbar(im)
plt.show()


plt.title("compensate_phasemask")
plt.hist(compensate_phasemask)
plt.show()

print(compensate_phasemask.min())
print(compensate_phasemask.max())

compensate_phasemask_normalized_255 = (compensate_phasemask * 255)

plt.title("compensate_phasemask_normalized_255")
im=plt.imshow(compensate_phasemask_normalized_255,cmap='gray')
plt.colorbar(im)
plt.show()

plt.title("compensate_phasemask_normalized_255")
plt.hist(compensate_phasemask_normalized_255)
plt.show()

# save image
output_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\compensate_phasemask.png'
cv2.imwrite(output_path, compensate_phasemask_normalized_255)

Vortex test

In [ ]:
# predicted_phasemask_original_size_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Gaussian\Abb08\results_phasemask_2\predicted_phasemask_original_size.png'
# original_phasemask_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Gaussian\Abb08\phasemask\phase_mask_2.png'

predicted_phasemask_original_size_path= r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex_Wrong_Phasemasks\evaluation_phasemask_predicted_original_size\predicted_phasemask_original_size_2.png'
original_phasemask_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\Evaluation_Vortex_Wrong_Phasemasks\phasemask\phase_mask_2.png'

predicted_phasemask_original_size = cv2.imread(predicted_phasemask_original_size_path)
original_phasemask = cv2.imread(original_phasemask_path)



predicted_phasemask_original_size = rgb2gray(predicted_phasemask_original_size)
original_phasemask = rgb2gray(original_phasemask)


plt.title("Beam_cropped")
im=plt.imshow(predicted_phasemask_original_size,cmap='gray')
plt.colorbar(im)
plt.show()

plt.title("Beam_cropped")
im=plt.imshow(original_phasemask,cmap='gray')
plt.colorbar(im)
plt.show()

print(predicted_phasemask_original_size.max())
print(original_phasemask.max())
print(predicted_phasemask_original_size.min())
print(original_phasemask.min())

compensate_phasemask = abs(original_phasemask - predicted_phasemask_original_size)
#compensate_phasemask = original_phasemask - predicted_phasemask_original_size

# print(compensate_phasemask.max())
# print(compensate_phasemask.min())
#print(compensate_phasemask)

print(compensate_phasemask)

plt.title("compensate_phasemask")
im=plt.imshow(compensate_phasemask,cmap='gray')
plt.colorbar(im)
plt.show()


plt.title("compensate_phasemask")
plt.hist(compensate_phasemask)
plt.show()

print(compensate_phasemask.min())
print(compensate_phasemask.max())

compensate_phasemask_normalized_255 = (compensate_phasemask * 255)

plt.title("compensate_phasemask_normalized_255")
im=plt.imshow(compensate_phasemask_normalized_255,cmap='gray')
plt.colorbar(im)
plt.show()

plt.title("compensate_phasemask_normalized_255")
plt.hist(compensate_phasemask_normalized_255)
plt.show()

# save image
output_path = r'\\srvditz1\lac\Studenten\AE_VoE_Stud\Sven Burckhard\Predict_Phasemask\compensate_phasemask.png'
cv2.imwrite(output_path, compensate_phasemask_normalized_255)